# Flujo de trabajo con humanos en el bucle utilizando Microsoft Agent Framework

## 🎯 Objetivos de aprendizaje

En este cuaderno, aprenderás a implementar flujos de trabajo **con humanos en el bucle** utilizando el `RequestInfoExecutor` de Microsoft Agent Framework. Este poderoso patrón permite pausar los flujos de trabajo de IA para recopilar información humana, haciendo que tus agentes sean interactivos y otorgando a los humanos control sobre decisiones críticas.

## 🔄 ¿Qué es el humano en el bucle?

**Humano en el bucle (HITL)** es un patrón de diseño donde los agentes de IA pausan la ejecución para solicitar la entrada de un humano antes de continuar. Esto es esencial para:

- ✅ **Decisiones críticas** - Obtener aprobación humana antes de tomar acciones importantes
- ✅ **Situaciones ambiguas** - Permitir que los humanos aclaren cuando la IA tiene incertidumbre
- ✅ **Preferencias del usuario** - Preguntar a los usuarios que elijan entre múltiples opciones
- ✅ **Cumplimiento y seguridad** - Garantizar supervisión humana para operaciones reguladas
- ✅ **Experiencias interactivas** - Crear agentes conversacionales que respondan a la entrada del usuario

## 🏗️ Cómo funciona en Microsoft Agent Framework

El marco proporciona tres componentes clave para HITL:

1. **`RequestInfoExecutor`** - Un ejecutor especial que pausa el flujo de trabajo y emite un `RequestInfoEvent`
2. **`RequestInfoMessage`** - Clase base para cargas útiles de solicitud tipificadas enviadas a humanos
3. **`RequestResponse`** - Correlaciona las respuestas humanas con las solicitudes originales utilizando `request_id`

**Patrón de flujo de trabajo:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Nuestro ejemplo: Reserva de hotel con confirmación del usuario

Ampliaremos el flujo de trabajo condicional añadiendo confirmación humana **antes** de sugerir destinos alternativos:

1. El usuario solicita un destino (por ejemplo, "París")
2. `availability_agent` verifica si hay habitaciones disponibles
3. **Si no hay habitaciones** → `confirmation_agent` pregunta "¿Te gustaría ver alternativas?"
4. El flujo de trabajo **se pausa** utilizando `RequestInfoExecutor`
5. **El humano responde** "sí" o "no" a través de la entrada de la consola
6. `decision_manager` decide según la respuesta:
   - **Sí** → Mostrar destinos alternativos
   - **No** → Cancelar la solicitud de reserva
7. Mostrar el resultado final

¡Esto demuestra cómo dar a los usuarios control sobre las sugerencias del agente!

---

¡Comencemos! 🚀


## Paso 1: Importar las bibliotecas necesarias

Importamos los componentes estándar del marco de trabajo de agentes además de las **clases específicas para intervención humana**:
- `RequestInfoExecutor` - Ejecutor que pausa el flujo de trabajo para la entrada humana
- `RequestInfoEvent` - Evento emitido cuando se solicita entrada humana
- `RequestInfoMessage` - Clase base para cargas útiles de solicitud tipificadas
- `RequestResponse` - Correlaciona las respuestas humanas con las solicitudes
- `WorkflowOutputEvent` - Evento para detectar las salidas del flujo de trabajo


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Paso 2: Definir modelos Pydantic para salidas estructuradas

Estos modelos definen el **esquema** que los agentes devolverán. Mantenemos todos los modelos del flujo de trabajo condicional y añadimos:

**Nuevo para Human-in-the-Loop:**
- `HumanFeedbackRequest` - Subclase de `RequestInfoMessage` que define la carga útil de la solicitud enviada a los humanos
  - Contiene `prompt` (pregunta a realizar) y `destination` (contexto sobre la ciudad no disponible)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Paso 3: Crear la Herramienta de Reservas de Hotel

La misma herramienta del flujo de trabajo condicional: verifica si hay habitaciones disponibles en el destino.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Paso 4: Definir funciones de condición para el enrutamiento

Necesitamos **cuatro funciones de condición** para nuestro flujo de trabajo con intervención humana:

**Del flujo de trabajo condicional:**
1. `has_availability_condition` - Redirige cuando hay hoteles disponibles
2. `no_availability_condition` - Redirige cuando NO hay hoteles disponibles

**Nuevo para la intervención humana:**
3. `user_wants_alternatives_condition` - Redirige cuando el usuario dice "sí" a las alternativas
4. `user_declines_alternatives_condition` - Redirige cuando el usuario dice "no" a las alternativas


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Paso 5: Crear el Ejecutor del Gestor de Decisiones

¡Este es el **núcleo del patrón de humano en el bucle**! El `DecisionManager` es un `Executor` personalizado que:

1. **Recibe retroalimentación humana** a través de objetos `RequestResponse`
2. **Procesa la decisión del usuario** (sí/no)
3. **Dirige el flujo de trabajo** enviando mensajes a los agentes correspondientes

Características clave:
- Utiliza el decorador `@handler` para exponer métodos como pasos del flujo de trabajo
- Recibe `RequestResponse[HumanFeedbackRequest, str]` que contiene tanto la solicitud original como la respuesta del usuario
- Genera mensajes simples de "sí" o "no" que activan nuestras funciones de condición


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Paso 6: Crear un Ejecutor de Visualización Personalizado

El mismo ejecutor de visualización del flujo de trabajo condicional: genera resultados finales como salida del flujo de trabajo.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Paso 7: Cargar Variables de Entorno

Configura el cliente LLM (Modelos de GitHub, Azure OpenAI o OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Paso 8: Crear Agentes de IA y Ejecutores

Creamos **seis componentes del flujo de trabajo**:

**Agentes (envueltos en AgentExecutor):**
1. **availability_agent** - Verifica la disponibilidad de hoteles usando la herramienta
2. **confirmation_agent** - 🆕 Prepara la solicitud de confirmación humana
3. **alternative_agent** - Sugiere ciudades alternativas (cuando el usuario dice que sí)
4. **booking_agent** - Fomenta la reserva (cuando hay habitaciones disponibles)
5. **cancellation_agent** - 🆕 Maneja el mensaje de cancelación (cuando el usuario dice que no)

**Ejecutores Especiales:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` que pausa el flujo de trabajo para obtener entrada humana
7. **decision_manager** - 🆕 Ejecutor personalizado que dirige según la respuesta humana (ya definido anteriormente)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Paso 9: Construir el flujo de trabajo con intervención humana

Ahora construimos el gráfico del flujo de trabajo con **enrutamiento condicional**, incluyendo el camino con intervención humana:

**Estructura del flujo de trabajo:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Conexiones clave:**
- `availability_agent → confirmation_agent` (cuando no hay habitaciones)
- `confirmation_agent → prepare_human_request` (tipo de transformación)
- `prepare_human_request → request_info_executor` (pausa para intervención humana)
- `request_info_executor → decision_manager` (siempre - proporciona RequestResponse)
- `decision_manager → alternative_agent` (cuando el usuario dice "sí")
- `decision_manager → cancellation_agent` (cuando el usuario dice "no")
- `availability_agent → booking_agent` (cuando hay habitaciones disponibles)
- Todos los caminos terminan en `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Paso 10: Ejecutar Caso de Prueba 1 - Ciudad SIN Disponibilidad (París con Confirmación Humana)

Este test demuestra el **ciclo completo con intervención humana**:

1. Solicitar hotel en París
2. availability_agent verifica → No hay habitaciones
3. confirmation_agent crea una pregunta dirigida al usuario
4. request_info_executor **pausa el flujo de trabajo** y emite `RequestInfoEvent`
5. **La aplicación detecta el evento y solicita al usuario en la consola**
6. El usuario escribe "sí" o "no"
7. La aplicación envía la respuesta mediante `send_responses_streaming()`
8. decision_manager decide según la respuesta
9. Se muestra el resultado final

**Patrón Clave:**
- Usar `workflow.run_stream()` para la primera iteración
- Usar `workflow.send_responses_streaming(pending_responses)` para las iteraciones posteriores
- Escuchar `RequestInfoEvent` para detectar cuándo se necesita la entrada del usuario
- Escuchar `WorkflowOutputEvent` para capturar los resultados finales


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Paso 11: Ejecutar Caso de Prueba 2 - Ciudad CON Disponibilidad (Estocolmo - No se necesita intervención humana)

Esta prueba demuestra el **camino directo** cuando hay habitaciones disponibles:

1. Solicitar hotel en Estocolmo  
2. availability_agent verifica → Habitaciones disponibles ✅  
3. booking_agent sugiere reservar  
4. display_result muestra confirmación  
5. **¡No se requiere intervención humana!**

El flujo evita completamente el camino con intervención humana cuando hay habitaciones disponibles.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Puntos clave y mejores prácticas para humanos en el bucle

### ✅ Lo que has aprendido:

#### 1. **Patrón RequestInfoExecutor**
El patrón de humanos en el bucle en Microsoft Agent Framework utiliza tres componentes clave:
- `RequestInfoExecutor` - Pausa el flujo de trabajo y emite eventos
- `RequestInfoMessage` - Clase base para cargas útiles de solicitud tipadas (¡subclasea esto!)
- `RequestResponse` - Correlaciona las respuestas humanas con las solicitudes originales

**Comprensión crítica:**
- `RequestInfoExecutor` NO recopila entradas por sí mismo - solo pausa el flujo de trabajo
- Tu código de aplicación debe escuchar `RequestInfoEvent` y recopilar entradas
- Debes llamar a `send_responses_streaming()` con un diccionario que asocie `request_id` con la respuesta del usuario

#### 2. **Patrón de ejecución en streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Arquitectura basada en eventos**
Escucha eventos específicos para controlar el flujo de trabajo:
- `RequestInfoEvent` - Se necesita entrada humana (flujo de trabajo pausado)
- `WorkflowOutputEvent` - El resultado final está disponible (flujo de trabajo completo)
- `WorkflowStatusEvent` - Cambios de estado (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Ejecutores personalizados con @handler**
El `DecisionManager` demuestra cómo crear ejecutores que:
- Usan el decorador `@handler` para exponer métodos como pasos del flujo de trabajo
- Reciben mensajes tipados (por ejemplo, `RequestResponse[HumanFeedbackRequest, str]`)
- Dirigen el flujo de trabajo enviando mensajes a otros ejecutores
- Acceden al contexto a través de `WorkflowContext`

#### 5. **Enrutamiento condicional con decisiones humanas**
Puedes crear funciones condicionales que evalúen respuestas humanas:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Aplicaciones en el mundo real:

1. **Flujos de trabajo de aprobación**
   - Obtener aprobación del gerente antes de procesar informes de gastos
   - Requerir revisión humana antes de enviar correos electrónicos automatizados
   - Confirmar transacciones de alto valor antes de ejecutarlas

2. **Moderación de contenido**
   - Marcar contenido cuestionable para revisión humana
   - Pedir a los moderadores que tomen decisiones finales en casos límite
   - Escalar a humanos cuando la confianza de la IA sea baja

3. **Servicio al cliente**
   - Permitir que la IA maneje preguntas rutinarias automáticamente
   - Escalar problemas complejos a agentes humanos
   - Preguntar al cliente si desea hablar con un humano

4. **Procesamiento de datos**
   - Pedir a los humanos que resuelvan entradas de datos ambiguas
   - Confirmar interpretaciones de IA de documentos poco claros
   - Permitir a los usuarios elegir entre múltiples interpretaciones válidas

5. **Sistemas críticos para la seguridad**
   - Requerir confirmación humana antes de acciones irreversibles
   - Obtener aprobación antes de acceder a datos sensibles
   - Confirmar decisiones en industrias reguladas (salud, finanzas)

6. **Agentes interactivos**
   - Crear bots conversacionales que hagan preguntas de seguimiento
   - Diseñar asistentes que guíen a los usuarios a través de procesos complejos
   - Diseñar agentes que colaboren con humanos paso a paso

### 🔄 Comparación: Con vs Sin humanos en el bucle

| Característica | Flujo de trabajo condicional | Flujo de trabajo con humanos en el bucle |
|----------------|------------------------------|------------------------------------------|
| **Ejecución** | Único `workflow.run()` | Bucle con `run_stream()` + `send_responses_streaming()` |
| **Entrada del usuario** | Ninguna (totalmente automatizado) | Indicaciones interactivas mediante `input()` o UI |
| **Componentes** | Agentes + Ejecutores | + RequestInfoExecutor + DecisionManager |
| **Eventos** | Solo AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Pausas** | Sin pausas | El flujo de trabajo se pausa en RequestInfoExecutor |
| **Control humano** | Sin control humano | Los humanos toman decisiones clave |
| **Caso de uso** | Automatización | Colaboración y supervisión |

### 🚀 Patrones avanzados:

#### Múltiples puntos de decisión humana
Puedes tener múltiples nodos `RequestInfoExecutor` en el mismo flujo de trabajo:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Manejo de tiempos de espera
Implementa tiempos de espera para respuestas humanas:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integración con UI enriquecida
En lugar de `input()`, integra con UI web, Slack, Teams, etc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Humanos en el bucle condicional
Solicita entrada humana solo en situaciones específicas:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Mejores prácticas:

1. **Siempre subclasea RequestInfoMessage**
   - Proporciona seguridad de tipo y validación
   - Permite un contexto enriquecido para la representación en UI
   - Clarifica la intención de cada tipo de solicitud

2. **Usa indicaciones descriptivas**
   - Incluye contexto sobre lo que estás preguntando
   - Explica las consecuencias de cada elección
   - Mantén las preguntas simples y claras

3. **Maneja entradas inesperadas**
   - Valida las respuestas de los usuarios
   - Proporciona valores predeterminados para entradas inválidas
   - Da mensajes de error claros

4. **Rastrea IDs de solicitud**
   - Usa la correlación entre request_id y respuestas
   - No intentes gestionar el estado manualmente

5. **Diseña para no bloquear**
   - No bloquees hilos esperando entradas
   - Usa patrones asíncronos en todo momento
   - Soporta instancias concurrentes de flujo de trabajo

### 📚 Conceptos relacionados:

- **Middleware de agentes** - Intercepta llamadas de agentes (notebook anterior)
- **Gestión del estado del flujo de trabajo** - Persiste el estado del flujo de trabajo entre ejecuciones
- **Colaboración entre múltiples agentes** - Combina humanos en el bucle con equipos de agentes
- **Arquitecturas basadas en eventos** - Construye sistemas reactivos con eventos

---

### 🎓 ¡Felicidades!

¡Has dominado los flujos de trabajo con humanos en el bucle usando Microsoft Agent Framework! Ahora sabes cómo:
- ✅ Pausar flujos de trabajo para recopilar entrada humana
- ✅ Usar RequestInfoExecutor y RequestInfoMessage
- ✅ Manejar ejecución en streaming con eventos
- ✅ Crear ejecutores personalizados con @handler
- ✅ Dirigir flujos de trabajo basados en decisiones humanas
- ✅ Construir agentes de IA interactivos que colaboren con humanos

**¡Este es un patrón crítico para construir sistemas de IA confiables y controlables!** 🚀



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
